In [ ]:
#Upload your kaggle API json file:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pramodh941","key":"8a743bc4d1f00db1f052ef456a100e62"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d stackoverflow/stacksample

 99% 1.10G/1.11G [00:18<00:00, 91.9MB/s]
100% 1.11G/1.11G [00:18<00:00, 64.2MB/s]


In [ ]:
#Define the path and name of the directory to be created:
#This is the path where we will extract the dataset:
import os
path = "/content/SS"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /content/SS 


In [ ]:
#Extracts the Images from the Zip file downloaded to the path we created above:
from zipfile import ZipFile
file_name = "stacksample.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall(path = "/content/SS/")
  print('Done')

Done


In [ ]:
#If you face issue loading the necessary java files:
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic

In [ ]:
!sudo apt-get install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [ ]:
user_schema = 'Id INT,OwnerUserId INT,CreationDate TIMESTAMP,ClosedDate TIMESTAMP,Score INT,Title STRING,Body STRING'
df = spark.read.option("header","true").csv('/content/SS/Questions.csv', schema = user_schema, sep=',', dateFormat='MM/dd/yyyy',timestampFormat='yyyy-MM-dd HH:mm:ss.SSSSSS') #.limit(100000)

In [ ]:
df.show()

+----+-----------+-------------------+----------+-----+--------------------+--------------------+
|  Id|OwnerUserId|       CreationDate|ClosedDate|Score|               Title|                Body|
+----+-----------+-------------------+----------+-----+--------------------+--------------------+
|  80|         26|2008-08-01 13:57:07|      null|   26|SQLStatement.exec...|"<p>I've written ...|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|  

In [ ]:
columns_to_drop = ['Id','OwnerUserId', 'CreationDate', 'ClosedDate','Score', 'Title']
df = df.drop(*columns_to_drop)

In [ ]:
df.show()

+--------------------+
|                Body|
+--------------------+
|"<p>I've written ...|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
+--------------------+
only showing top 20 rows



In [ ]:
df.filter("Body is null").count()

31745443

In [ ]:
df = df.filter(df.Body.isNotNull())

In [ ]:
df.count()

1404281

In [ ]:
df.show()

+--------------------+
|                Body|
+--------------------+
|"<p>I've written ...|
|"<p>Are there any...|
|<p>Has anyone got...|
|<p>This is someth...|
|<p>I have a littl...|
|<p>I am working o...|
|<p>I've been writ...|
|<p>I wonder how y...|
|<p>I would like t...|
|<p>I'm trying to ...|
| and haven't seen...|
|<p>What's the sim...|
|<p>I need to grab...|
|<p>I'm looking fo...|
|<p>What is the co...|
|<p>I am using CCN...|
|<p>I am looking t...|
|<p>I am using MSB...|
|<p>I'm setting up...|
|<p>I always creat...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import col, lower, regexp_replace, split

def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "<p>", "")
  c = regexp_replace(c, "</p>", "")
  c = regexp_replace(c, "<code>", "")
  c = regexp_replace(c, "</code>", "")
  c = regexp_replace(c, "<strong>", "")
  c = regexp_replace(c, "</strong>", "")
  c = regexp_replace(c, "'ve", "")
  c = regexp_replace(c, "'m", "")
  c = regexp_replace(c, "'s", "")
  c = regexp_replace(c, "'t", "")
  c = regexp_replace(c, "<[^>]+>", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "^rt ", "")
  return c

In [ ]:
clean_text_df = df.select(clean_text(col("Body")).alias("Body"))

In [ ]:
clean_text_df.show(30)

+--------------------+
|                Body|
+--------------------+
|i written a datab...|
|are there any rea...|
|has anyone got ex...|
|this is something...|
|i have a little g...|
|i am working on a...|
|i been writing a ...|
|i wonder how you ...|
|i would like the ...|
|i trying to maint...|
| and haven seen u...|
|what the simplest...|
|i need to grab th...|
|i looking for a w...|
|what is the corre...|
|i am using ccnet ...|
|i am looking to a...|
|i am using msbuil...|
|i setting up a de...|
|i always create a...|
|if i adding a col...|
|getting back into...|
|is it possible to...|
|postgresql is int...|
|i want to get the...|
|i currently use a...|
|i want to be able...|
|how do you disabl...|
|what are good lib...|
|a quick glance at...|
+--------------------+
only showing top 30 rows



In [ ]:
from pyspark.sql.functions import split, col
array_clean_text_df = clean_text_df.withColumn("Body", split(col("Body"), "\s+").cast("array<string>")) #in cast change as required data type suitable for the methods we use

In [ ]:
array_clean_text_df.dtypes

[('Body', 'array<string>')]

In [ ]:
array_clean_text_df.show()

+--------------------+
|                Body|
+--------------------+
|[i, written, a, d...|
|[are, there, any,...|
|[has, anyone, got...|
|[this, is, someth...|
|[i, have, a, litt...|
|[i, am, working, ...|
|[i, been, writing...|
|[i, wonder, how, ...|
|[i, would, like, ...|
|[i, trying, to, m...|
|[, and, haven, se...|
|[what, the, simpl...|
|[i, need, to, gra...|
|[i, looking, for,...|
|[what, is, the, c...|
|[i, am, using, cc...|
|[i, am, looking, ...|
|[i, am, using, ms...|
|[i, setting, up, ...|
|[i, always, creat...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import StopWordsRemover 

In [ ]:
stopwordList = ["really","are","good","anyone","someone","got","something","little","working","wonder","guys","like","seen","way","grab","looking","using","always","haven","allow","doesn","don","need","possible","want","able","quick","glance","useful","simple","note","make","wondering","problem","today","tomorrow","let","getting","following","say","idea","thing","currently"] 
stopwordList.extend(StopWordsRemover().getStopWords())
#stopwordList = list(set(stopwordList))#optionnal
remover = StopWordsRemover(inputCol="Body", outputCol="BodyCleaned" ,stopWords=stopwordList)

In [ ]:
CleanedQuestions = remover.transform(array_clean_text_df)

In [ ]:
CleanedQuestions.show()

+--------------------+--------------------+
|                Body|         BodyCleaned|
+--------------------+--------------------+
|[i, written, a, d...|[written, databas...|
|[are, there, any,...|[tutorials, expla...|
|[has, anyone, got...|[experience, crea...|
|[this, is, someth...|[pseudosolved, ma...|
|[i, have, a, litt...|[game, written, c...|
|[i, am, working, ...|[collection, clas...|
|[i, been, writing...|[writing, web, se...|
|[i, wonder, how, ...|[manage, deployme...|
|[i, would, like, ...|[version, propert...|
|[i, trying, to, m...|[trying, maintain...|
|[, and, haven, se...|[, usermachine, h...|
|[what, the, simpl...|[simplest, connec...|
|[i, need, to, gra...|[base64encoded, r...|
|[i, looking, for,...|[delete, file, lo...|
|[what, is, the, c...|[correct, get, pr...|
|[i, am, using, cc...|[ccnet, sample, p...|
|[i, am, looking, ...|[users, control, ...|
|[i, am, using, ms...|[msbuild, build, ...|
|[i, setting, up, ...|[setting, dedicat...|
|[i, always, creat...|[create, n

In [ ]:
from pyspark.sql.functions import concat_ws

In [ ]:
concatDF = CleanedQuestions.withColumn("BodyCleaned", concat_ws(",",col("BodyCleaned")))

In [ ]:
concatDF.show()

+--------------------+--------------------+
|                Body|         BodyCleaned|
+--------------------+--------------------+
|[i, written, a, d...|written,database,...|
|[are, there, any,...|tutorials,explain...|
|[has, anyone, got...|experience,creati...|
|[this, is, someth...|pseudosolved,many...|
|[i, have, a, litt...|game,written,c,us...|
|[i, am, working, ...|collection,classe...|
|[i, been, writing...|writing,web,servi...|
|[i, wonder, how, ...|manage,deployment...|
|[i, would, like, ...|version,property,...|
|[i, trying, to, m...|trying,maintain,s...|
|[, and, haven, se...|,usermachine,hive...|
|[what, the, simpl...|simplest,connect,...|
|[i, need, to, gra...|base64encoded,rep...|
|[i, looking, for,...|delete,file,locke...|
|[what, is, the, c...|correct,get,proce...|
|[i, am, using, cc...|ccnet,sample,proj...|
|[i, am, looking, ...|users,control,sub...|
|[i, am, using, ms...|msbuild,build,stu...|
|[i, setting, up, ...|setting,dedicated...|
|[i, always, creat...|create,new

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
tokenizer = Tokenizer(inputCol="BodyCleaned", outputCol="words")

In [ ]:
wordsData = tokenizer.transform(concatDF)

In [ ]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=8)

In [ ]:
featurizedData = hashingTF.transform(wordsData)

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [ ]:
idfModel = idf.fit(featurizedData)

In [ ]:
TfIdfData = idfModel.transform(featurizedData)

In [ ]:
TfIdfData.show()

+--------------------+--------------------+--------------------+-------------+--------------------+
|                Body|         BodyCleaned|               words|  rawFeatures|            features|
+--------------------+--------------------+--------------------+-------------+--------------------+
|[i, written, a, d...|written,database,...|[written,database...|(8,[4],[1.0])|(8,[4],[1.9793253...|
|[are, there, any,...|tutorials,explain...|[tutorials,explai...|(8,[4],[1.0])|(8,[4],[1.9793253...|
|[has, anyone, got...|experience,creati...|[experience,creat...|(8,[3],[1.0])|(8,[3],[2.0447428...|
|[this, is, someth...|pseudosolved,many...|[pseudosolved,man...|(8,[4],[1.0])|(8,[4],[1.9793253...|
|[i, have, a, litt...|game,written,c,us...|[game,written,c,u...|(8,[6],[1.0])|(8,[6],[2.1047877...|
|[i, am, working, ...|collection,classe...|[collection,class...|(8,[3],[1.0])|(8,[3],[2.0447428...|
|[i, been, writing...|writing,web,servi...|[writing,web,serv...|(8,[5],[1.0])|(8,[5],[2.1133317...|


In [ ]:
TfIdfData.dtypes

[('Body', 'array<string>'),
 ('BodyCleaned', 'array<string>'),
 ('rawFeatures', 'vector'),
 ('features', 'vector')]

In [ ]:
from pyspark.ml.feature import MinHashLSH

In [ ]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
LSHmodel = mh.fit(TfIdfData)

In [ ]:
LSH = LSHmodel.transform(TfIdfData)

In [ ]:
LSH.show()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|                Body|         BodyCleaned|               words|  rawFeatures|            features|              hashes|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|[i, written, a, d...|written,database,...|[written,database...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|
|[are, there, any,...|tutorials,explain...|[tutorials,explai...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|
|[has, anyone, got...|experience,creati...|[experience,creat...|(8,[3],[1.0])|(8,[3],[2.0447428...|[[2.2946146E7], [...|
|[this, is, someth...|pseudosolved,many...|[pseudosolved,man...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|
|[i, have, a, litt...|game,written,c,us...|[game,written,c,u...|(8,[6],[1.0])|(8,[6],[2.1047877...|[[1.604125001E9],...|
|[i, am, working, ...|collection

In [ ]:
lists = TfIdfData.select('features').take(1)

In [ ]:
lists

[Row(features=SparseVector(8, {4: 1.9793}))]

In [ ]:
from pyspark.ml.linalg import Vectors
key = Vectors.sparse(8,[4],[1.9793])

In [ ]:
LSHmodel.approxNearestNeighbors(TfIdfData, key, 10).show()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+
|                Body|         BodyCleaned|               words|  rawFeatures|            features|              hashes|distCol|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+
|[i, written, a, d...|written,database,...|[written,database...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|    0.0|
|[are, there, any,...|tutorials,explain...|[tutorials,explai...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|    0.0|
|[this, is, someth...|pseudosolved,many...|[pseudosolved,man...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|    0.0|
|[what, the, simpl...|simplest,connect,...|[simplest,connect...|(8,[4],[1.0])|(8,[4],[1.9793253...|[[1.229364012E9],...|    0.0|
|[i, need, to, gra...|base64encoded,rep...|[base64encoded,re...|(8,[4],[1.0])|(8,[4],[1.9793253..

In [ ]:
LSH.select('words').take(1)

[Row(words=['written,database,generation,script,sql,execute,adobe,air,application'])]